<a href="https://colab.research.google.com/github/radanim/TIL/blob/master/TCN_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load the Packages

In [ ]:
!pip install keras-tcn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from tensorflow.keras import Model, Input
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from tcn import TCN

### Define the max features

In [ ]:
max_features = 20000

In [ ]:
maxlen = 100
batch_size = 32

### Load the IMDB Dataset

In [ ]:
print("Loading Data...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

### Print the shape of dataset

In [ ]:
print(f"X_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

### Pad Sequence(Samples X Time)
* Pad Sequence added and see the shape for X 

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
print(f"X_train shape: {x_train.shape}")
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print(f"X_test shape: {x_test.shape}")

* Convert target y into Numpy Array 

In [ ]:
y_train = np.array(y_train)
print(f"y_train shape: {y_train.shape}")
y_test = np.array(y_test)
print(f"y_test shape: {y_test.shape}")

### Design the TCN Model

In [ ]:
i = Input(shape = (maxlen,))
x = Embedding(max_features, 128)(i)
x = TCN(nb_filters = 64, kernel_size=6, dilations=[1,2,4,8,16,32,64])(x)
x = Dropout(0.5)(x)
x = Dense(1, activation = 'sigmoid')(x)
model = Model(inputs=[i], outputs=[x])
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics = ['accuracy'])
model.summary()

### Model Training

In [ ]:
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = 10, validation_data = [x_test, y_test])

In [ ]:
result = model.evaluate(x_test, y_test)
print(f"Accuracy : {result[1] * 100:.2f} %")
print(f"Loss : {result[0] * 100:.2f} %")

In [ ]:
from tensorflow import keras
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [ ]:
yhat_probs = model.predict(x_test, verbose=0)
print(y_test)
print(np.argmax(yhat_probs, axis=1))

In [ ]:
# predict probabilities for test set
yhat_probs = np.argmax(model.predict(x_test, verbose=0), axis=1)
 
# kappa
kappa = cohen_kappa_score(y_test, yhat_probs)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(y_test, yhat_probs)
print('ROC AUC: %f' % auc)

print(classification_report(y_test, yhat_probs))
# confusion matrix
matrix = confusion_matrix(y_test, yhat_probs)
sns.heatmap(matrix, annot=True)
